In [ ]:
%%qsharp
%azure.connect "/subscriptions/2f8b131d-6276-4fe0-be2d-4646cd04042a/resourceGroups/classicalcomputing/providers/Microsoft.Quantum/Workspaces/classicalcomputing" location="eastus"


In [2]:
import qsharp

Preparing Q# environment...
..

In [3]:
%%qsharp

open Microsoft.Quantum.Intrinsic;
open Microsoft.Quantum.Diagnostics;
open Microsoft.Quantum.Convert;
open Microsoft.Quantum.Math;
open Microsoft.Quantum.Measurement;

operation QFTImpl (qs : Qubit[]) : Unit is Adj + Ctl
{
    body (...)
    {
        let nQubits = Length(qs);

        for i in 0 .. nQubits - 1
        {
            H(qs[i]);
            for j in i + 1 .. nQubits - 1
            {
                Controlled R1Frac([qs[j]], (1, j - i, qs[i]));
            }
        }

        Microsoft.Quantum.Canon.SwapReverseRegister(qs);
    }
}

In [4]:
%%qsharp

operation QuantumAdd (x : Qubit[], y : Qubit[]) : Unit is Adj + Ctl {
    let n = Length(x);
    QFTImpl(y);
    for i in 0 .. n - 1 {
        for j in 0 .. (n - 1) - i {
            Controlled R1Frac([x[i + j]], (2, j + 1, (y)[(n - 1) - i]));
        }
    }
    Adjoint QFTImpl(y);
}

operation QuantumAddByNumber (x : Qubit[], b : Int) : Unit is Adj + Ctl {
    let n = Length(x);

    // apply Draper adder for numeric
    QFTImpl(x);
    for i in 0 .. n - 1 {
        for j in 0 .. (n - 1) - i {
            if(not((b / 2^((n - 1) - (i + j))) % 2 == 0)) {
                R1Frac(2, j + 1, (x)[(n - 1) - i]);
            }
        }
    }
    Adjoint QFTImpl(x);
}

operation QuantumAddByModulus (N : Int, x : Qubit[], y : Qubit[]) : Unit is Adj + Ctl {
    use (ancilla, cx, cy) = (Qubit(), Qubit(), Qubit()) {
        // add bit for preventing overflow
        let x_large = [cx] + x;
        let y_large = [cy] + y;
        // |x⟩ |y⟩ -> |x⟩ |x + y⟩
        QuantumAdd(x_large, y_large);
        // |y⟩ -> |y - N⟩
        Adjoint QuantumAddByNumber(y_large, N);
        // Turn on ancilla when first bit is |1⟩ (i.e, when x + y - N < 0)
        Controlled X([y_large[0]], ancilla);
        // Add N back when ancilla is |1⟩
        Controlled QuantumAddByNumber([ancilla], (y_large, N));
        // set ancilla to |0⟩
        Adjoint QuantumAdd(x_large, y_large);
        X(ancilla);
        Controlled X([y_large[0]], ancilla);
        QuantumAdd(x_large, y_large);
    }
}

operation QuantumMultiplyByModulus (N : Int, a : Int, y : Qubit[]) : Unit is Adj + Ctl {
    let n = Length(y);
    let a_mod = a % N;

    use s = Qubit[n] {
        // start |y⟩ |0⟩

        // apply adder by repeating "a" (integer) times
        for r in 0 .. a_mod - 1 {
            QuantumAddByModulus(N, y, s);
        }
        // now |y⟩ |a y mod N⟩

        // swap first register and second one by tuple
        Microsoft.Quantum.Canon.ApplyToEachCA(SWAP, Microsoft.Quantum.Arrays.Zipped(y, s));
        // now |a y mod N⟩ |y⟩

        // reset all s qubits !
        // but it's tricky because we cannot use "Reset()" since here is controlled operator.
        let a_inv = InverseModI(a_mod, N);
        for r in 0 .. a_inv - 1 {
            Adjoint QuantumAddByModulus(N, y, s);
        }
    }
}

In [5]:
%%qsharp

operation QuantumExponentForPeriodFinding (a : Int, N : Int, x : Qubit[], y : Qubit[]) : Unit {
    let n1 = Length(x);
    let n2 = Length(y);

    // set |y⟩ = |0...01⟩
    X(y[n2 - 1]);

    for idx in 0 .. n1 - 1 {
        // a^(2^((n1-1) - idx)) is too big, then we reduce beforehand
        mutable a_mod = 1;
        for power in 1 .. 2^((n1-1) - idx) {
            set a_mod = (a_mod * a) % N;
        }
        // apply decomposition elements
        Controlled QuantumMultiplyByModulus([x[idx]], (N, a_mod, y));
    }
}

In [6]:
%%qsharp

operation QuantumPeriodFinding(num : Int, a : Int) : Int {
    // Get least integer n1 such as : num^2 <= 2^n1
    let n1 = BitSizeI(num) * 2;
    let n2 = BitSizeI(num);
    mutable periodCandidate = 1;
    repeat {
        use (x, y) = (Qubit[n1], Qubit[n2]) {
            Microsoft.Quantum.Canon.ApplyToEachCA(H, x);

            // |x⟩ |0 (=y)⟩ -> |x⟩ |a^x mod N⟩
            QuantumExponentForPeriodFinding(a, num, x, y);

            // measure y and reset
            mutable tmpResult = new Result[n2];
            for idx in 0 .. n2 - 1 {
                set tmpResult w/= idx <-MResetZ(y[idx]);
            }

            // QFT for x
            QFTImpl(x);

            // Measure x and reset
            mutable realResult = new Result[n1];
            for idx in 0 .. n1 - 1 {
                set realResult w/= idx <-MResetZ(x[idx]);
            }
            
            // get integer's result from measured array (ex : |011⟩ -> 3)
            let resultBool = [false] + Microsoft.Quantum.Convert.ResultArrayAsBoolArray(realResult); // for making unsigned positive integer, add first bit
            let resultBool_R = Microsoft.Quantum.Arrays.Reversed(resultBool); // because BoolArrayAsBigInt() is Little Endian order
            let resultIntL = Microsoft.Quantum.Convert.BoolArrayAsBigInt(resultBool_R);

            // get period candidate by continued fraction expansion (thanks to Euclid !)
            let gcdL = GreatestCommonDivisorL(resultIntL, 2L^n1);
            let calculatedNumerator = resultIntL / gcdL;
            let calculatedDenominator = 2L^n1 / gcdL;
            let numL = Microsoft.Quantum.Convert.IntAsBigInt(num);
            let approximatedFraction =
                ContinuedFractionConvergentL(BigFraction(calculatedNumerator, calculatedDenominator), numL);
            let (approximatedNumerator, approximatedDenominator) = approximatedFraction!;
            mutable periodCandidateL = 0L;
            if(approximatedDenominator < 0L) {
                set periodCandidateL = approximatedDenominator * -1L;
            }
            else {
                set periodCandidateL = approximatedDenominator;             
            }
            set periodCandidate = ReduceBigIntToInt(periodCandidateL);

            // output for debugging
            Message($"Measured Fraction : {resultIntL} / {2L^n1}");
            Message($"Approximated Fraction : {approximatedNumerator} / {approximatedDenominator}");
            Message($"Period Candidate : {periodCandidate}");
        }
    }
    until ((periodCandidate != 0) and (ExpModI(a, periodCandidate, num) == 1))
    fixup {
    }

    // output for debugging
    Message("Found period " + Microsoft.Quantum.Convert.IntAsString(periodCandidate));
    Message("");
    return periodCandidate;
}

// This is a helper function to convert BigInt to Int
operation ReduceBigIntToInt(numL : BigInt) : Int {
    // Check if numL is not large
    Microsoft.Quantum.Diagnostics.Fact(BitSizeL(numL) <= 32, $"Cannot convert to Int. Input is too large");

    mutable resultInt = 0;
    let numArray = Microsoft.Quantum.Convert.BigIntAsBoolArray(numL);
    let numArray_R = Microsoft.Quantum.Arrays.Reversed(numArray); // because BigIntAsBoolArray() is Little Endian order
    let nSize = Length(numArray_R);
    for idx in 0 .. nSize - 1 {
        if(numArray_R[idx] and ((nSize - 1) - idx <= 31)) {
            set resultInt = resultInt + (2 ^ ((nSize - 1) - idx));
        }
    }
    return resultInt;
}

/snippet_.qs(15,33): warning QS3308: Deprecated syntax. Use [] to construct an empty array, or [x, size = n] to construct an array of x repeated n times.
/snippet_.qs(24,34): warning QS3308: Deprecated syntax. Use [] to construct an empty array, or [x, size = n] to construct an array of x repeated n times.


In [7]:
def ordem_multiplicativa_n (periodo):
    ord_mult_n = periodo//2
    return ord_mult_n

def fator_n (ord_mult_n, a, n):
    fator = gcd(((a**ord_mult_n)-1), n)
    return fator

In [9]:
# Implementação quântica do algoritmo de Encontrar o Período Quântico
# Suponha que a simulação seja realizada e o período seja armazenado em "periodo_encontrado"

# Valores de exemplo
a = 3
n = 6

calculatedPeriodCandidate = QuantumPeriodFinding.simulate(num=n, a=a)

# Calculando a ordem multiplicativa usando o período encontrado
'''ord_mult_n = ordem_multiplicativa_n(calculatedPeriodCandidate)

# Calculando o fator n
fator = fator_n(ord_mult_n, a, n)

print("Período encontrado:", QuantumPeriodFinding)
print("Ordem multiplicativa de n:", ord_mult_n)
print("Fator n:", fator)'''
